In [ ]:
from web_to_note import extract_main_content, generate_single_note_from_urls

# Test extract_main_content function
test_url = "https://en.wikipedia.org/wiki/Cursor_(code_editor)"
print("Testing extract_main_content...")
content = extract_main_content(test_url)
if content:
    print("Content extracted successfully:")
    print(content[:500])  # Print the first 500 characters for brevity
else:
    print("Failed to extract content.")


In [ ]:
# Test generate_single_note_from_urls function
test_urls = [
    test_url
]
test_note_name = "Cursor CTO Interview"
generate_single_note_from_urls(test_urls, test_note_name, verbose=True)

In [ ]:
urls = [
    'https://en.wikipedia.org/wiki/Cursor_(code_editor)',
    'https://www.youtube.com/watch?v=4jDQi9P9UIw'
]

In [ ]:
# test the youtube transcription tool

from youtube_transcript_api import YouTubeTranscriptApi
from text_to_note import obsidify_text

youtube_url = "https://www.youtube.com/watch?v=4jDQi9P9UIw"

yt_id = youtube_url.split("v=")[1].split("&")[0]
transcript = YouTubeTranscriptApi.get_transcript(yt_id)

transcript_text = " ".join([t["text"] for t in transcript])
obsidify_text(transcript_text, "Cursor CTO Interview", ignore_token_limit=True)

In [ ]:
# test the youtube transcription tool

from youtube_transcript_api import YouTubeTranscriptApi
from text_to_note import obsidify_text
from web_to_note import extract_main_content

url = 'https://www.youtube.com/watch?v=LCEmiRjPEtQ'

generate_single_note_from_urls([url], "Andrej Karpathy Software Is Changing (Again)", insert_links=True, verbose=True)

In [ ]:
#test the note insertion
from text_to_note import insert_links_to_existing_notes
from tools.md_files import get_notes_list

note_list = get_notes_list()


with open('sample_note.md', 'r') as f:
    sample_note_text = f.read()


print('length of sample note text: ', len(sample_note_text))
print(sample_note_text[:1000])

In [ ]:
#long note list to test O() complexity
import random
import string

long_note_list = [
    ''.join(random.choices(string.ascii_letters + string.digits, k=10)) 
    for _ in range(100000)
]
long_note_list[:10]

In [ ]:
import time

# Timing the function for note_list
start_time = time.time()
updated_sample_note = insert_links_to_existing_notes(sample_note_text, note_list)
end_time = time.time()
print(f"Time taken for note_list: {end_time - start_time:.6f} seconds")

# Timing the function for long_note_list
start_time = time.time()
updated_sample_note_long = insert_links_to_existing_notes(sample_note_text, long_note_list)
end_time = time.time()
print(f"Time taken for long_note_list: {end_time - start_time:.6f} seconds")



In [ ]:
import random
import string

for mag in range(10):
    start_time = time.time()

    long_note_list = [
        ''.join(random.choices(string.ascii_letters + string.digits, k=10)) 
        for _ in range(10**mag)
    ]
    end_time = time.time()
    print(f"Length of long_note_list: {len(long_note_list)}")
    print(f"Time taken: {end_time - start_time:.6f} seconds")

In [ ]:
# test the note name validation

from text_to_note import assert_note_name_is_valid

test_names = [
    "Cursor CTO Interview",
    'test:notename',
    'budget[2025]',
    'turing-NLG',
    '#1 #2 heck yeah']

for name in test_names:
    try:
        assert_note_name_is_valid(name+".md")
        print(f"Note name '{name}.md' is valid.")
    except ValueError as e:
        print(f"Note name '{name}.md' is invalid: {e}")

In [ ]:
from tools.brave_search import get_web_links

results = get_web_links("curl command", count=10)
results

In [1]:
from tools.brave_search import get_web_content
import dspy
from dotenv import load_dotenv
import os
load_dotenv()

api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = 'gpt-4.1'


azure_lm = dspy.LM(
    model=f"azure/{deployment_name}",
    api_key=api_key,
    api_base=api_base,
    api_version="2023-03-15-preview"
)


dspy.configure(lm=azure_lm)


/Users/Ben.Wyatt/Repos/personal/obsidian-llm-tool-use/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class WebResearcher(dspy.Signature):
    """You are a researcher who can search the web for information."""
    question: str = dspy.InputField(description="The question that needs to be answered.")
    answer: list[str] = dspy.OutputField(description="The answer from the web.")

agent = dspy.ReAct(
    WebResearcher,
    tools=[get_web_content]
)
    

In [5]:
result = agent(question="What is the name of the criterion collection movie that is a documentary about a middle eastern country like Lebanon or Palestine?")

Error fetching URL https://www.criterion.com/current/posts/6395-a-leading-palestinian-auteur-looks-toward-the-light: 403 Client Error: Forbidden for url: https://www.criterion.com/current/posts/6395-a-leading-palestinian-auteur-looks-toward-the-light


In [6]:
result

Prediction(
    trajectory={'thought_0': 'I need to identify a Criterion Collection documentary that focuses on a Middle Eastern country such as Lebanon or Palestine. To do this, I should search for a list of Criterion Collection documentaries about Middle Eastern countries, or specifically about Lebanon or Palestine, to find the relevant title.', 'tool_name_0': 'get_web_content', 'tool_args_0': {'query': 'Criterion Collection documentary about Lebanon or Palestine', 'count': 5}, 'observation_0': 'Source URL: https://www.filmslop.com/features/does-the-criterion-collection-care-about-arab-cinema\n\nContent:\nThree years ago, I discovered what would become one of my favorite movies: Nabil Maleh’s The Extras (1993). The Syrian comedy-drama follows Salem, an aspiring actor who can only find work as an extra, trying to arrange a secret meeting with Nada, the girl he is courting. The two can only be together in public under the supervision of Nada’s strict family—it is against the law for un

In [ ]:
result.trajectory

In [ ]:
dspy.inspect_history(n=10)

## How to Expand this:
- replace the basic web flow with this react agent
- make it an orchestration: the question-answerer has tools that trigger web search agents and other types of agents too
  - like what? 